In [1]:
from pyspark.sql import SparkSession 
spark=SparkSession.builder.appName('rc').getOrCreate()

22/08/19 22:19:55 WARN Utils: Your hostname, faouzi-Inspiron-15-3567 resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface wlp2s0)
22/08/19 22:19:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/19 22:20:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/19 22:20:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import *

In [3]:
df=spark.read.csv('movie_ratings_df.csv',inferSchema=True,header=True)

In [4]:
print((df.count(),len(df.columns)))

(100000, 3)


In [5]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



In [6]:
# showing a sample of the dataframe
df.orderBy(rand()).show(10,False)

+------+--------------------------------------+------+
|userId|title                                 |rating|
+------+--------------------------------------+------+
|724   |Picture Perfect (1997)                |3     |
|867   |Trainspotting (1996)                  |5     |
|727   |Aladdin (1992)                        |4     |
|749   |Snow White and the Seven Dwarfs (1937)|5     |
|543   |Fish Called Wanda, A (1988)           |3     |
|399   |Fargo (1996)                          |3     |
|450   |Sneakers (1992)                       |5     |
|921   |Babe (1995)                           |3     |
|586   |Star Trek: The Wrath of Khan (1982)   |3     |
|268   |Mortal Kombat (1995)                  |3     |
+------+--------------------------------------+------+
only showing top 10 rows



In [7]:
#check number of ratings by each user
df.groupBy('userId').count().orderBy('count',ascending=False).show(11,False)

+------+-----+
|userId|count|
+------+-----+
|405   |737  |
|655   |685  |
|13    |636  |
|450   |540  |
|276   |518  |
|416   |493  |
|537   |490  |
|303   |484  |
|234   |480  |
|393   |448  |
|181   |435  |
+------+-----+
only showing top 11 rows



In [8]:
#check number of ratings by each user
df.groupBy('userId').count().orderBy('count',ascending=True).show(10,False)

+------+-----+
|userId|count|
+------+-----+
|19    |20   |
|143   |20   |
|309   |20   |
|34    |20   |
|202   |20   |
|732   |20   |
|441   |20   |
|685   |20   |
|824   |20   |
|631   |20   |
+------+-----+
only showing top 10 rows



In [9]:
#number of times movie been rated 
df.groupBy('title').count().orderBy('count',ascending=False).show(10,False)

+-----------------------------+-----+
|title                        |count|
+-----------------------------+-----+
|Star Wars (1977)             |583  |
|Contact (1997)               |509  |
|Fargo (1996)                 |508  |
|Return of the Jedi (1983)    |507  |
|Liar Liar (1997)             |485  |
|English Patient, The (1996)  |481  |
|Scream (1996)                |478  |
|Toy Story (1995)             |452  |
|Air Force One (1997)         |431  |
|Independence Day (ID4) (1996)|429  |
+-----------------------------+-----+
only showing top 10 rows



In [11]:
from pyspark.ml.feature import StringIndexer,IndexToString

In [12]:
#convert the movie title columns into numerical values
stringIndexer = StringIndexer(inputCol="title", outputCol="title_new")

In [13]:
model = stringIndexer.fit(df)

In [14]:
#creating a new dataframe the numerical values
indexed = model.transform(df)

In [15]:
#validate the numerical title values
indexed.show(10)

+------+------------+------+---------+
|userId|       title|rating|title_new|
+------+------------+------+---------+
|   196|Kolya (1996)|     3|    287.0|
|    63|Kolya (1996)|     3|    287.0|
|   226|Kolya (1996)|     5|    287.0|
|   154|Kolya (1996)|     3|    287.0|
|   306|Kolya (1996)|     5|    287.0|
|   296|Kolya (1996)|     4|    287.0|
|    34|Kolya (1996)|     5|    287.0|
|   271|Kolya (1996)|     4|    287.0|
|   201|Kolya (1996)|     4|    287.0|
|   209|Kolya (1996)|     4|    287.0|
+------+------------+------+---------+
only showing top 10 rows



In [17]:
#split the data into training and test datatset with 75% , 25% ratio
train,test=indexed.randomSplit([0.75,0.25])

In [18]:
# number of rows in the training set
train.count()

75238

In [19]:
#number of rows in the testing set
test.count()

24762

In [20]:
from pyspark.ml.recommendation import ALS

In [21]:
#Training a new recommender model using the training datatset
rec=ALS(maxIter=10,regParam=0.01,userCol='userId',itemCol='title_new',ratingCol='rating',nonnegative=True,coldStartStrategy="drop")

In [22]:
#fit the model on the training set
rec_model=rec.fit(train)

In [23]:
#making predictions on test set 
predicted_ratings=rec_model.transform(test)

In [24]:
#columns in predicted ratings dataframe
predicted_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- title_new: double (nullable = false)
 |-- prediction: float (nullable = false)



In [25]:
#predicted vs actual ratings for test set 
predicted_ratings.orderBy(rand()).show(10)

+------+--------------------+------+---------+----------+
|userId|               title|rating|title_new|prediction|
+------+--------------------+------+---------+----------+
|   747|Empire Strikes Ba...|     5|     18.0|  4.733433|
|   618|       Casino (1995)|     3|    367.0| 3.0412626|
|   230|      Top Gun (1986)|     5|     94.0|  4.081959|
|   459| Bed of Roses (1996)|     4|    534.0| 3.5929885|
|   330|Room with a View,...|     5|    235.0| 4.7540293|
|   709|Amityville Horror...|     4|    579.0| 2.5730107|
|   804|Bride of Frankens...|     3|    629.0| 3.4041326|
|   429| Howling, The (1981)|     3|    727.0| 2.9746141|
|   542|This Is Spinal Ta...|     4|    127.0| 4.0259337|
|   451|       Scream (1996)|     5|      6.0| 3.4882598|
+------+--------------------+------+---------+----------+
only showing top 10 rows



In [26]:
#importing Regression Evaluator to measure RMSE
from pyspark.ml.evaluation import RegressionEvaluator

In [27]:
#create Regressor evaluator object for measuring accuracy
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')

In [29]:
#apply the RE on predictions dataframe to calculate RMSE
rmse=evaluator.evaluate(predicted_ratings)

In [30]:
# RMSE error is higher in case of smaller datasets
print(rmse)

1.022350573809048


In [31]:
# Recommending movies to users

In [40]:
#create dataset of all distinct movies 
unique_movies=indexed.select('title_new').distinct()
unique_movies.count()

1664

In [34]:
a = unique_movies.alias('a')

In [35]:
user_id=85

In [36]:
#movies already watched and rated 
watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new').distinct()
watched_movies.count()

In [38]:
b=watched_movies.alias('b')

In [41]:
#joining both tables on a left join 
total_movies = a.join(b, a.title_new == b.title_new,how='left')
total_movies.show(10,False)

In [43]:
# movies not yet rated
remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
remaining_movies.count()

In [45]:
#adding new column of user_Id of active useer to remaining movies df 
remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
remaining_movies.show(10,False)

In [46]:
#making recommendations using ALS recommender model and selecting only top 'n' movies
recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False)
recommendations.show(5,False)

In [48]:
#restitution of string names for movie titles
movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)
final_recommendations=movie_title.transform(recommendations)
final_recommendations.show(10,False)

+---------+------+----------+------------------------------------------------------+
|title_new|userId|prediction|title                                                 |
+---------+------+----------+------------------------------------------------------+
|1347.0   |85    |6.0238442 |Angel Baby (1995)                                     |
|1411.0   |85    |5.558659  |Boys, Les (1997)                                      |
|1040.0   |85    |5.161837  |Boys of St. Vincent, The (1993)                       |
|302.0    |85    |4.578575  |Close Shave, A (1995)                                 |
|1348.0   |85    |4.4852552 |Ballad of Narayama, The (Narayama Bushiko) (1958)     |
|488.0    |85    |4.4800076 |Wallace & Gromit: The Best of Aardman Animation (1996)|
|880.0    |85    |4.4090953 |Jude (1996)                                           |
|1410.0   |85    |4.395792  |Bitter Sugar (Azucar Amargo) (1996)                   |
|108.0    |85    |4.3956304 |Rear Window (1954)                  

In [49]:
# function to recommend top 'n' movies to any particular user from the dataframe
def Recommend(user_id,n):
    
    a = unique_movies.alias('a')
    
    watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new')
    
    b=watched_movies.alias('b')
    
    total_movies = a.join(b, a.title_new == b.title_new,how='left')
    
    remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
    
    remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
    
    recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False).limit(n)    
    
    movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)
    final_recommendations=movie_title.transform(recommendations)
    
    return final_recommendations.show(n,False)

In [51]:
Recommend(55,15)

+---------+------+----------+-------------------------------------------------------------------------------+
|title_new|userId|prediction|title                                                                          |
+---------+------+----------+-------------------------------------------------------------------------------+
|928.0    |55    |16.016726 |Paradise Lost: The Child Murders at Robin Hood Hills (1996)                    |
|1139.0   |55    |14.541603 |Warriors of Virtue (1997)                                                      |
|1342.0   |55    |13.563986 |8 Heads in a Duffel Bag (1997)                                                 |
|999.0    |55    |13.452765 |Stuart Saves His Family (1995)                                                 |
|1289.0   |55    |12.705158 |World of Apu, The (Apur Sansar) (1959)                                         |
|1263.0   |55    |12.185088 |City of Industry (1997)                                                        |
|943.0    